# Ceneo Scraper

## Struktura opinii w serwisie Ceneo.pl

|składowa|selektor|zmienna|
|--------|--------|-------|
|identyfikator opinii|["data-entry-id"]|opinion_id|
|autora|.user-post__author-name|author|
|rekomendację|span.user-post__author-recomendation > em|recommend|
|liczbę gwiazdek|span.user-post__score-count|stars|
|data wystawienia opinii|span.user-post__published > time:nth-child(1)["datetime"]|opinion_date|
|data zakupu produktu|span.user-post__published > time:nth-child(2)["datetime"]|purchase_date|
|ile osób uznało opinię za przydatną|button.vote-yes["data-total-vote"]|up_vote|
|ile osób uznało opinię za nieprzydatną|button.vote-no["data-total-vote"]|down_vote|
|treść opinii|div.user-post__text|content|
|listę wad|div.review-feature__title review-feature__title--positives ~ div.review-feature__item|cons|
|listę zalet|div.review-feature__title review-feature__title--negatives ~ div.review-feature__item|pros|

In [65]:
selectors = {
"opinion_id": (None,"data-entry-id"),
"author": (".user-post__author-name",),
"recommend": ("span.user-post__author-recomendation > em",),
"stars": ("span.user-post__score-count",),
"content": ("div.user-post__text",),
"cons": ("div.review-feature__title review-feature__title--positives ~ div.review-feature__item",None,True),
"pros": ("div.review-feature__title review-feature__title--negatives ~ div.review-feature__item",None,True),
"opinion_date": ("span.user-post__published > time:nth-child(1)","datetime"),
"purchase_date": ("span.user-post__published > time:nth-child(2)","datetime"),
"up_vote": ("button.vote-yes","data-total-vote"),
"down_vote": ("button.vote-no","data-total-vote")
}

## Funckja do ekstrakcji danych z kodu HTML

In [66]:
def  extract(ancestor, selector=None, attribute=None, return_list = False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select(selector)]
        return [tag.text.strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip()

1. Import bibliotek

In [67]:
import os
import json
import requests
from bs4 import BeautifulSoup

2. wczytanie kodu produktu o którym maja zostać pobrane opinie

In [68]:
# product_id = '151210226'
product_id = input("Podaj kod produktu: ")
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"


3. Wysłanie do serwera Ceneo.pl zadania dostępu do strony

In [69]:
response = requests.get(url)

In [70]:
response.text

'\r\n\r\n<!DOCTYPE html>\r\n<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7  non-direct ab-inactive" lang="pl"> <![endif]-->\r\n<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8  non-direct ab-inactive" lang="pl"> <![endif]-->\r\n<!--[if IE 8]>         <html class="no-js lt-ie9  non-direct ab-inactive" lang="pl"> <![endif]-->\r\n<!--[if gt IE 8]><!-->\r\n<html class="no-js  non-direct ab-inactive" lang="pl">\r\n<!--<![endif]-->\r\n<head>\r\n    <meta charset="utf-8">\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n    <link rel="preconnect" href="https://image.ceneo.pl">\n<link rel="preconnect" href="https://image.ceneostatic.pl">\n<link rel="preconnect" href="https://www.google-analytics.com">\n<link rel="preconnect" href="https://www.googleadservices.com">\n<link rel="preconnect" href="https://adservice.google.com">\n<link rel="preconnect" href="https://www.google.pl">\n<link rel="preconnect" href="https://www.google.com">\n<link rel="preconnect" 

4. Przekstałcenie kodu HTML w postaci tekstowej do obiektu reprezentującego strukturę DOM
5. Wydobrycie ze strutktury DOM znaczników odpowiadających opiniom konsumentów

6. Wydobycie z struktury znacznika odpowiadającego pojedynczej opinii jej składowych

3. Pobranie wszystkich opinii o wskazanym produkcie z serwisu Ceneo.pl

In [71]:
all_opinions = []
while(url):
    response = requests.get(url)
    page = BeautifulSoup(response.text,"html.parser")
    opinions = page.select("div.js_product-review")
    for opinion in opinions:
        single_opinion = {
            key: extract(opinion,*value)
                for key, value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:
        url = "https://www.ceneo.pl" + extract(page,"a.pagination__next","href")
    except TypeError:
        url = None

4. Zapis opinii o produkcie do pliku JSON

In [72]:
if not os.path.exists('opinions'):
    os.mkdir('opinions')
with open(f'opinions/{product_id}.json',"w",encoding="UTF-8") as jf:
    json.dump(all_opinions,jf,indent=4, ensure_ascii=False)